<a href="https://colab.research.google.com/github/drf0rk/AnxLight/blob/main/notebook/AnxLight_Launcher_v0.0.9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Cell 1: AnxLight v3 Pre-Flight Setup (Robust)
# This cell ensures the repository is up-to-date and runs the main environment setup with enhanced logging and error handling.

import os
import subprocess
import sys
import time
import platform
from datetime import datetime

# --- Enhanced Logging ---
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_dir = f"/content/anxlight_logs_{timestamp}"
os.makedirs(log_dir, exist_ok=True)
log_file = f"{log_dir}/preflight_setup.log"

def log_message(message, level="INFO"):
    """Log a message to both console and log file"""
    timestamp_msg = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    formatted_msg = f"[{timestamp_msg}] [{level}] {message}"
    print(formatted_msg)
    with open(log_file, "a") as f:
        f.write(formatted_msg + "\n")

# --- Initial System Checks ---
log_message(f"Starting AnxLight v3 Pre-Flight Setup")
log_message(f"Python version: {sys.version}")
log_message(f"Platform: {platform.platform()}")

try:
    git_version = subprocess.check_output(["git", "--version"], text=True).strip()
    log_message(f"Git version: {git_version}")
except Exception as e:
    log_message(f"Git not found or not working properly: {e}", "ERROR")
    sys.exit(1)

# --- Step 1: Clone/Update the Repository ---
repo_path = '/content/AnxLight'
if not os.path.exists(repo_path):
    log_message(f"Cloning AnxLight repository to {repo_path}...")
    try:
        result = subprocess.run(
            ['git', 'clone', 'https://github.com/drf0rk/AnxLight.git', repo_path],
            check=True, capture_output=True, text=True, timeout=120
        )
        log_message(f"Clone successful.")
    except subprocess.TimeoutExpired:
        log_message("Git clone operation timed out after 120 seconds", "ERROR")
        sys.exit(1)
    except subprocess.CalledProcessError as e:
        log_message(f"FATAL: Failed to clone repository. Error: {e.stderr}", "ERROR")
        sys.exit(1)
else:
    log_message("AnxLight repository already exists. Pulling latest changes...")
    try:
        # Discard any local changes and pull the latest from the repo for a clean state
        subprocess.run(['git', '-C', repo_path, 'reset', '--hard', 'origin/main'], check=True, capture_output=True, text=True, timeout=60)
        pull_result = subprocess.run(['git', '-C', repo_path, 'pull'], check=True, capture_output=True, text=True, timeout=60)
        log_message(f"Pull result: {pull_result.stdout.strip()}")
    except subprocess.TimeoutExpired:
        log_message("Git pull operation timed out. Continuing with existing state.", "WARNING")
    except subprocess.CalledProcessError as e:
        log_message(f"WARNING: Failed to pull latest changes: {e.stderr}", "WARNING")

# --- Step 2: Set Environment ---
os.chdir(repo_path)
log_message(f"Current working directory: {os.getcwd()}")
os.environ['PROJECT_ROOT'] = repo_path
os.environ['ANXLIGHT_LOG_DIR'] = log_dir
log_message(f"PROJECT_ROOT set to: {os.environ['PROJECT_ROOT']}")

# --- Step 3: Execute the Pre-Flight Setup Script ---
pre_flight_script_path = os.path.join(repo_path, 'scripts', 'pre_flight_setup.py')
python_executable = sys.executable

if not os.path.exists(pre_flight_script_path):
    log_message(f"FATAL: Pre-flight script not found at {pre_flight_script_path}", "ERROR")
    sys.exit(1)

log_message(f"Executing Pre-Flight Setup Script: {pre_flight_script_path}")
preflight_log = f"{log_dir}/preflight_script_output.log"
log_message(f"Detailed script logs will be saved to: {preflight_log}")

try:
    with open(preflight_log, 'w') as log_f:
        process = subprocess.Popen(
            [python_executable, pre_flight_script_path],
            stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True
        )
        # Stream output to both console and log file
        for line in iter(process.stdout.readline, ''):
            print(line, end='')
            log_f.write(line)

        return_code = process.wait()

        if return_code != 0:
            log_message(f"Pre-flight setup failed with return code {return_code}", "ERROR")
            sys.exit(1)

    log_message("Pre-Flight Setup Script Finished Successfully")

except Exception as e:
    log_message(f"FATAL: An unexpected error occurred during pre-flight setup: {e}", "ERROR")
    import traceback
    log_message(traceback.format_exc(), "ERROR")
    sys.exit(1)

log_message("Cell 1 completed successfully. You can now run Cell 2.")

[2025-06-26 19:12:14] [INFO] Starting AnxLight v3 Pre-Flight Setup
[2025-06-26 19:12:14] [INFO] Python version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
[2025-06-26 19:12:14] [INFO] Platform: Linux-6.1.123+-x86_64-with-glibc2.35
[2025-06-26 19:12:14] [INFO] Git version: git version 2.34.1
[2025-06-26 19:12:14] [INFO] AnxLight repository already exists. Pulling latest changes...
[2025-06-26 19:12:15] [INFO] Pull result: Updating c4943e6..f559c90
Fast-forward
 scripts/main_gradio_app.py | 10 +++++++---
 1 file changed, 7 insertions(+), 3 deletions(-)
[2025-06-26 19:12:15] [INFO] Current working directory: /content/AnxLight
[2025-06-26 19:12:15] [INFO] PROJECT_ROOT set to: /content/AnxLight
[2025-06-26 19:12:15] [INFO] Executing Pre-Flight Setup Script: /content/AnxLight/scripts/pre_flight_setup.py
[2025-06-26 19:12:15] [INFO] Detailed script logs will be saved to: /content/anxlight_logs_20250626_191214/preflight_script_output.log
--- AnxLight Pre-Flight Setup Script v0.1.15 ---

In [ ]:
# @title Cell 2: AnxLight v3 Gradio App Launcher (Robust)
# This cell launches the main Gradio UI with enhanced process management, logging, and error detection.

import os
import sys
import subprocess
import time
import signal
import atexit
from datetime import datetime

# --- Enhanced Logging ---
log_dir = os.environ.get('ANXLIGHT_LOG_DIR', f"/content/anxlight_logs_{datetime.now().strftime('%Y%m%d_%H%M%S')}")
os.makedirs(log_dir, exist_ok=True)
gradio_log_file = f"{log_dir}/gradio_app.log"

def log_message(message, level="INFO"):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    formatted_msg = f"[{timestamp}] [{level}] {message}"
    print(formatted_msg)
    with open(gradio_log_file, "a") as f:
        f.write(formatted_msg + "\n")

log_message("Starting AnxLight v3 Launch Cell")

# --- Path Verification ---
project_root = os.environ.get('PROJECT_ROOT', '/content/AnxLight')
venv_python_path = os.path.join(project_root, 'anxlight_venv', 'bin', 'python')
main_gradio_script_path = os.path.join(project_root, 'scripts', 'main_gradio_app.py')

if not os.path.exists(venv_python_path):
    log_message(f"CRITICAL ERROR: Virtual environment not found at {venv_python_path}. Please run Cell 1.", "ERROR")
    sys.exit(1)

# --- Process Management ---
gradio_process = None

def cleanup_process():
    global gradio_process
    if gradio_process and gradio_process.poll() is None:
        log_message("Terminating Gradio process...")
        try:
            gradio_process.terminate()
            gradio_process.wait(timeout=5)
            log_message("Gradio process terminated gracefully.")
        except subprocess.TimeoutExpired:
            gradio_process.kill()
            log_message("Gradio process forcefully killed.")
        except Exception as e:
            log_message(f"Error during cleanup: {e}", "ERROR")

atexit.register(cleanup_process)

# --- Launch Gradio App ---
gradio_env = os.environ.copy()
gradio_env['PYTHONPATH'] = f"{os.path.join(project_root, 'modules')}:{project_root}:{os.path.join(project_root, 'scripts')}:{gradio_env.get('PYTHONPATH', '')}"
log_message(f"Launching: {main_gradio_script_path}")
log_message(f"Log file: {gradio_log_file}")

try:
    gradio_process = subprocess.Popen(
        [venv_python_path, main_gradio_script_path],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True, env=gradio_env, cwd=project_root
    )
    log_message("Waiting for Gradio URL...")

    url_found = False
    start_time = time.time()
    timeout = 300 # 5 minutes
    url_patterns = ["Running on public URL:", "Public URL:"]

    for line in iter(gradio_process.stdout.readline, ''):
        print(line, end='')
        with open(gradio_log_file, "a") as f: f.write(line)

        if any(p in line for p in url_patterns):
            print("\n" + "="*60)
            print("          GRADIO IS RUNNING AND READY!")
            print("          Click the public URL above to open the UI.")
            print("="*60 + "\n")
            url_found = True
            break

        if time.time() - start_time > timeout:
            log_message(f"Timeout waiting for Gradio URL after {timeout} seconds.", "ERROR")
            break

    if url_found:
        # Stream the rest of the output indefinitely
        for line in iter(gradio_process.stdout.readline, ''):
            print(line, end='')
            with open(gradio_log_file, "a") as f: f.write(line)
    else:
        log_message("Gradio process finished or timed out before a URL was generated.", "ERROR")

except KeyboardInterrupt:
    log_message("Keyboard interrupt received. Shutting down.", "WARNING")
except Exception as e:
    log_message(f"An error occurred while launching Gradio: {e}", "ERROR")
finally:
    cleanup_process()
    log_message("Launch cell finished.")

[2025-06-26 19:13:30] [INFO] Starting AnxLight v3 Launch Cell
[2025-06-26 19:13:30] [INFO] Launching: /content/AnxLight/scripts/main_gradio_app.py
[2025-06-26 19:13:30] [INFO] Log file: /content/anxlight_logs_20250626_191214/gradio_app.log
[2025-06-26 19:13:30] [INFO] Waiting for Gradio URL...
